### Import required libraries

In [1]:
import numpy as np
import pandas as pd 
import os
import gc
import sklearn.model_selection
from sklearn.linear_model import LinearRegression
import lightgbm as lgb

## suppress warnings
import warnings
warnings.filterwarnings("ignore")

### Process Whole Data

In [3]:
csv_path = "/export/storage_cpoliqui/data/raw/wages2008.tsv"
data = pd.read_csv(csv_path, sep = '\t', header = 0)

header_raw = ["Firm_Identifier", "Establishment_Identifier", "Year", "emp_present_12_31",
              "Worker_Identifier", "Avg_Month_Wage_In_miniWage", "contract_salary_rea", "salary_type",
              "Weekly_hrs", "Occupation_code", "reason_leave", "How_worker_entered", "date_enter",
              "date_left", "employment_type", "tenure_in_month", "female", "Education_level", "nationality",
              "Race", "Age", "municipality", "Microregion", "Corp_Form", "Industry_cnae"]

data.columns = header_raw

def transform_sal(sal_type, Weekly_hrs, contract_salary_rea):
    map_a = {1:4.333, 2:2.1667, 3:1, 4:0.2, 5:(1 / Weekly_hrs)}
    return contract_salary_rea / (Weekly_hrs * map_a[sal_type])

v_transform_sal = np.vectorize(transform_sal)

def process_data(data_1):
    data_1.drop(columns ="Microregion",inplace=True)
    data_1.drop(columns ="date_left",inplace=True)
    data_1.drop(columns ="date_enter",inplace=True)
    mode_race = data_1["Race"].mode()
    data_1["Race"].fillna(mode_race, inplace=True)
    data_1.dropna(inplace = True)
    
    data_1["contract_salary_rea"]=data_1["contract_salary_rea"]/100
    data_1=data_1.query('salary_type<=5')
    data_1=data_1.loc[(data_1["Firm_Identifier"]>=10)&(data_1["Race"]!=9)]
    data_1=data_1.loc[(data_1["contract_salary_rea"]>0)&(data_1["Avg_Month_Wage_In_miniWage"]>0)]   
    data_1["hr_salary"]=v_transform_sal(data_1["salary_type"].values,data_1["Weekly_hrs"].values,data_1["contract_salary_rea"].values)
    data_1.loc[:,"hr_salary_log"]=np.log(data_1["hr_salary"])
    
    #get rid of a small portion of entries where log(w)<0
    data_1=data_1.loc[(data_1["hr_salary_log"]>=0)]
    data_1=data_1.reset_index(drop=True)
    
    return data_1

data = process_data(data) # whole data

### Split single-market firms and multi-market firms

In [4]:
#set aside multi market firm from single market firm
def Single_Multiple(data_1):
    a=data_1.groupby(["Firm_Identifier"])["municipality"].nunique()
    single_market_id=a.loc[a==1]
    multi_market_id=a.loc[a!=1]
    multi_market_id=pd.Series(multi_market_id.index)
    single_market_id=pd.Series(single_market_id.index)

    single_market_firm=data_1.loc[data_1['Firm_Identifier'].isin(single_market_id)]
    multi_market_firm=data_1.loc[data_1['Firm_Identifier'].isin(multi_market_id)]
    return single_market_firm, multi_market_firm

single_whole, multi_whole = Single_Multiple(data) # whole data

### For single-market firms, split a 10% training set and a 90% testing set

In [5]:
def get_train_test(data_1):
    train, test = sklearn.model_selection.train_test_split(data_1, train_size=0.1,
                                                            test_size=0.9, random_state=42)
    print('Train:', train.shape, 'Test:', test.shape)    
    train.reset_index(drop=True,inplace=True)
    test.reset_index(drop=True,inplace=True)
    return train, test

single_train, single_test = get_train_test(single_whole)

def design_matrix(t):
    result = t.drop(columns=["Year", "Worker_Identifier", "Avg_Month_Wage_In_miniWage",
                    "contract_salary_rea", "hr_salary", "emp_present_12_31"])
    result.reset_index(drop=True,inplace=True)
    return result

def rmse(pred, actual):
    return np.sqrt(np.mean((actual - pred) ** 2))

Train: (3016748, 24) Test: (27150734, 24)


### Transform data into LightGBM format

In [6]:
single_train = design_matrix(single_train)
multi_whole = design_matrix(multi_whole)
single_test = design_matrix(single_test)

### Find the best set of hyper-parameters for the LightGBM model

In [7]:
def tune_lgb(data,                   ## data input, including all predictors and the outcome
             x_var,                  ## an array predictor names
             y_var,                  ## outcome name
             param_arr,              ## an array of hyper-parameter sets, in the form of dictionaries
             k_fold = 10,            ## the k for k-fold cross-validation
             test_size = 0.33,       ## size pf the test size for cross-validation
             random_state = 127,     ## set a random seed for the train-test splitting
             verbose_eval = False):  ## False to turn off log output when fitting the LightGBM
    
    ## import required libraries
    import pandas as pd
    import numpy as np
    import lightgbm as lgb
    import sklearn.model_selection
    
    def rmse(pred, actual):
        return np.sqrt(np.mean((actual - pred) ** 2))
    
    
    ## seperate predictors(x) and outcome(y)
    ## if no specific columns names for x, then use all variables
    if len(x_var) == 0:
        x = data.drop(columns = y_var)
        x.reset_index(drop = True, inplace = True)
    else:
        x = data[x_var].drop(columns = y_var)
        x.reset_index(drop = True, inplace = True)
    
    y = data[y_var]

    mean_rmse = []
    params_final = []
    
    ## test each set of input hyper-parameteres
    for par in param_arr:
        
        params = {'boosting_type': 'gbdt',
                  'objective': 'regression',
                  'metric': 'rmse',
                  'max_depth': -1,
                  'learning_rate': 0.1,
                  'early_stopping_round': 20,
                  'num_leaves': 50,
                  'verbose': -1,
                  'num_iterations': 10000}
        
        params.update(par)
        params_final.append(params)
        
        iter_rmse = []
        
        ## k-fold cross-validation
        for i in range(0, k_fold - 1):
            
            x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y,
                                                                       test_size = test_size,
                                                                       random_state = random_state + i)
            ## tranform data into LightGBM format
            x_train_lgb = lgb.Dataset(x_train, label = y_train)
            
            ## fit the model using the training set
            model = lgb.train(params, x_train_lgb, valid_sets = x_train_lgb,
                              verbose_eval = verbose_eval)
            
            ## predict using the testing set
            pred = model.predict(x_test)
            
            ## store the testing RMSE
            iter_rmse.append(rmse(pred.reshape(1, -1)[0], y_test))
        
        mean_rmse.append(np.mean(iter_rmse))
    
    ind = mean_rmse.index(np.min(mean_rmse))  ## the index of the smallest average RMSE
    
    print("A minimum mean testing RMSE of", mean_rmse[ind], "is given by parameter set", ind + 1)
    print(param_arr[ind])
    
    return params_final[ind] ## return a complete set of the best hyper-parameters


## Try some arbitury sets of hyper-parameters

### Get the best hyper-parameters

In [8]:
params1 = {'learning_rate': 0.05, "num_leaves": 30}
params2 = {'learning_rate': 0.1, "num_leaves": 30}
params3 = {'learning_rate': 0.2, "num_leaves": 30}
params4 = {'learning_rate': 0.05, "num_leaves": 50}
params5 = {'learning_rate': 0.1, "num_leaves": 50}
params6 = {'learning_rate': 0.2, "num_leaves": 50}
params7 = {'learning_rate': 0.05, "num_leaves": 70}
params8 = {'learning_rate': 0.1, "num_leaves": 70}
params9 = {'learning_rate': 0.2, "num_leaves": 70}


best_params = tune_lgb(data = single_train, x_var = [], y_var = "hr_salary_log",
                       param_arr = [params1,params2,params3,params4,params5,params6,params7,params8,params9],
                       k_fold = 2)

print(best_params)

A minimum mean testing RMSE of 0.33897898369152873 is given by parameter set 8
{'learning_rate': 0.1, 'num_leaves': 70}
{'boosting_type': 'gbdt', 'objective': 'regression', 'metric': 'rmse', 'max_depth': -1, 'learning_rate': 0.1, 'early_stopping_round': 20, 'num_leaves': 70, 'verbose': -1, 'num_iterations': 10000}


### Fit the model with the 10% training set of single-market firms

In [9]:
single_train_lgb = lgb.Dataset(single_train, label = single_train["hr_salary_log"])

model_lgb = lgb.train(best_params, single_train_lgb, valid_sets = single_train_lgb,
                      verbose_eval = False)

### Predict using the 90% testing set of single-market firms and the entire multi-market firms data

In [10]:
pred_single = model_lgb.predict(single_test).reshape(1, -1)   ## single-market firms, 90% testing set
pred_multi = model_lgb.predict(multi_whole).reshape(1, -1)    ## multi-market firms, entire data


print(rmse(pred_single[0], single_test["hr_salary_log"]))

print(rmse(pred_multi[0], multi_whole["hr_salary_log"]))

0.030469906518112218
0.049330518095784266
